In [1]:
import sys
sys.path.insert(0, "../..")

# Overview


In this notebook, we'll look at a few use cases showing how ConText can be used to extract information from clinical text.

In [2]:
import spacy
import medspacy

from medspacy.visualization import visualize_dep, visualize_ent

from medspacy.context import ConTextRule, ConTextComponent
from medspacy.ner import TargetRule

# 1. Classifying documents as positive or negative for pneumonia
In this example, we'll use ConText to assert whether mentions of pneumonia are experienced are not. We'll then infer whether an entire document is positive or negative for pneumonia.

In [3]:
nlp = medspacy.load()

In [4]:
nlp.pipe_names

['medspacy_pyrush', 'medspacy_target_matcher', 'medspacy_context']

In [5]:
texts = [
         "No radiographic evidence of pneumonia.",
         "Likely CHF, infiltrate.",
        "IMPRESSION:  Left lower lobe pneumonia.",
         "Possible consolidation.",
        
]

Let's first define patterns to match all of the phrases related to pneumonia. We'll use the medspacy `TargetMatcher` component to extract these as entities in the pipeline and then define rules using `TargetRule`.

In [6]:
target_matcher = nlp.get_pipe("medspacy_target_matcher")

In [7]:
target_rules = [
    TargetRule("pneumonia", "EVIDENCE_OF_PNEUMONIA", pattern=[{'LOWER': {'REGEX': 'pneumonias?'}}]),
    TargetRule("pneumonia", "EVIDENCE_OF_PNEUMONIA", pattern=[{'LOWER': {'REGEX': 'pna'}}]),
    TargetRule("consolidation", "EVIDENCE_OF_PNEUMONIA"),
    TargetRule("EVIDENCE_OF_PNEUMONIA", "EVIDENCE_OF_PNEUMONIA", pattern=[{'LOWER': {'REGEX': 'infiltrat(e|es|ion)'}}]),
          ]

In [8]:
target_matcher.add(target_rules)

Now let's process our docs and check which are determined to be non-positive:

In [9]:
docs = list(nlp.pipe(texts))

In [10]:
visualize_ent(docs[0])

In [11]:
visualize_ent(docs[1])

In [12]:
visualize_ent(docs[2])

In [13]:
visualize_ent(docs[3])

In [14]:
for doc in docs:
    print(doc)
    print("ent", "is_negated", "is_uncertain", sep="\t")
    for ent in doc.ents:
        print(ent, ent._.is_negated, ent._.is_uncertain, sep="\t")
    print()

No radiographic evidence of pneumonia.
ent	is_negated	is_uncertain
pneumonia	True	False

Likely CHF, infiltrate.
ent	is_negated	is_uncertain
infiltrate	False	True

IMPRESSION:  Left lower lobe pneumonia.
ent	is_negated	is_uncertain
pneumonia	False	False

Possible consolidation.
ent	is_negated	is_uncertain
consolidation	False	True



We have now asserted whether mentions of pneumonia are positive are negative. However, we are often interested in classifying pneumonia at a **document level**, not a span level. For example, although there are two mentions of pneumonia below and one is not experienced, the document overall is.

To do this, we'll apply some document inferencing logic. We'll then write a simple function which returns True if there is *at least one* mention of pneumonia which is definitively experienced by the patient. We'll also write a helper function to exclude mentions which are negated, uncertain, etc. 

We'll register a new extension, `Doc._.pneumonia_positive` and use this function as the getter.

This logic could also be put into a component as part of a processing pipeline.

In [15]:
from spacy.tokens import Doc

In [16]:
from medspacy.context.context_component import DEFAULT_ATTRS
DEFAULT_ATTRS

{'NEGATED_EXISTENCE': {'is_negated': True},
 'POSSIBLE_EXISTENCE': {'is_uncertain': True},
 'HISTORICAL': {'is_historical': True},
 'HYPOTHETICAL': {'is_hypothetical': True},
 'FAMILY': {'is_family': True}}

In [17]:
def get_pneumonia_positive(doc):
    """Return True if a doc contains at least one mention of 
    pneumonia which is not negated, uncertain, historical, 
    hypothetical, or experienced by family.
    """
    for ent in doc.ents:
        if ent.label_ != "EVIDENCE_OF_PNEUMONIA":
            continue
        if include_ent(ent):
#             print(ent)
            return True
    return False

def include_ent(ent):
    if ent._.is_negated:
        return False
    if ent._.is_uncertain:
        return False
    if ent._.is_historical:
        return False
    if ent._.is_hypothetical:
        return False
    if ent._.is_family:
        return False
    return True

In [18]:
docs

[No radiographic evidence of pneumonia.,
 Likely CHF, infiltrate.,
 IMPRESSION:  Left lower lobe pneumonia.,
 Possible consolidation.]

In [19]:
Doc.set_extension("pneumonia_positive", getter=get_pneumonia_positive, force=True)

In [20]:
pos_docs = [doc for doc in docs if doc._.pneumonia_positive is True]
neg_docs = [doc for doc in docs if doc._.pneumonia_positive is False]

In [21]:
pos_docs

[IMPRESSION:  Left lower lobe pneumonia.]

In [22]:
neg_docs

[No radiographic evidence of pneumonia.,
 Likely CHF, infiltrate.,
 Possible consolidation.]

# 2. Extracting anatomical sites of surgical site infections
So far, we've been using ConText to **assert** whether clinical conditions are actually present by checking for negation, indication, and family history. However, the ConText algorithm can be used to find other relationships between concepts in text. 

In this example, we'll show how ConText can be used to find the anatomical sites of surgical site infections, as done in [this paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5977582/). After extracting the target entities, we'll identify anatomical sites as modifiers and connect them to our targets.

In [23]:
nlp = medspacy.load(disable=["context"]) # We'll instantiate ConText later

In [24]:
nlp.pipe_names

['medspacy_pyrush', 'medspacy_target_matcher']

We'll start by extracting targets using the EntityRuler. The evidence of infections in these text are **"abscess"**, "**hematomas"**, and **"collection of fluid"**.

In [25]:
texts = ["There is abscess in the abdomen.",
        "There is a collection of fluid in the jejunum.",
        "hematomas are seen around the right lower quadrant"]

In [26]:
target_matcher = nlp.get_pipe("medspacy_target_matcher")

In [27]:
target_rules = [
    TargetRule("abscess", "EVIDENCE_OF_SSI"),
    TargetRule("hematomas", "EVIDENCE_OF_SSI"),
    TargetRule("collection of fluid", "EVIDENCE_OF_SSI"),
]

In [28]:
target_matcher.add(target_rules)

Now we'll instantiate context. This time, we'll set `add_attrs` to False because we aren't interested in looking for negation or temporality.

We'll now define ConTextItems which match the anatomical sites in the text. When matching, these anatomical sites will be treated the same way as the negation and indication modifiers above. If there are targets within the scope of the matched TagObjects, an edge will be created between them.

In [29]:
context = ConTextComponent(nlp, add_attrs=False)

In [30]:
rules = [
    ConTextRule(literal='abdomen', category='ANATOMICAL_SITE', direction='BIDIRECTIONAL'),
    ConTextRule(literal='jejunum', category='ANATOMICAL_SITE', direction='BIDIRECTIONAL'),
    ConTextRule(literal='right lower quadrant', category='ANATOMICAL_SITE', direction='BIDIRECTIONAL')
]

In [31]:
context.add(rules)

nlp.add_pipe("medspacy_context")

In [32]:
docs = list(nlp.pipe(texts))

../..\medspacy\common\medspacy_matcher.py:80: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc)
../..\medspacy\common\medspacy_matcher.py:80: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc)
../..\medspacy\common\medspacy_matcher.py:80: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc)


In [33]:
visualize_dep(docs[0])
visualize_ent(docs[0], colors={"ANATOMICAL_SITE": "#c3fca4",
                                  "EVIDENCE_OF_SSI": "orange"})

Now that we have our edges, we'll go a step further and add a new attribute to the targets called `anatomical_site`. If a target has a modifier of **"ANATOMICAL_SITE"**, we'll set this new attribute to be the text of the matched span 

In [34]:
from spacy.tokens import Span

In [35]:
Span.set_extension("anatomical_site", default=None, force=True)

In [36]:
for doc in docs:
    for ent in doc.ents:
        for mod in ent._.modifiers:
            if mod.category == 'ANATOMICAL_SITE':
                ent._.anatomical_site = mod.span.text
        print("{0} --> {1}".format(ent, ent._.anatomical_site))

abscess --> None
collection of fluid --> None
hematomas --> None


# 3. Family History of Breast Cancer
Another task for ConText might be identifying patients who have a family history of breast cancer. In this case, we want to first extract mentions of "breast cancer" - these will be our target concepts. We then need to define **"FAMILY_HISTORY"** modifiers, as well as any other semantic modifiers such as negation.

In [37]:
nlp = medspacy.load()

In [38]:
nlp.pipe_names

['medspacy_pyrush', 'medspacy_target_matcher', 'medspacy_context']

Here are our example texts. The first two are both positive for family history of breast cancer. The final two are negative: In one, the patient themself experiences breast cancer; in the last one, family history of cancer is explicitly negated.

In [39]:
texts = ["She has a family history of breast cancer.",
        "The pt's mother passed away of breast cancer several years ago.",
         "The patient was diagnosed with breast cancer in 2012.",
        "No family history of breast ca.",
]

We'll define one simple rule to match "breast cancer" and "breast ca" in our texts. We'll add these to an EntityRuler and add that to the pipeline.

In [40]:
target_rules = [
    TargetRule("breast cancer", "BREAST_CANCER", 
               pattern=[
                   {"LOWER": "breast"}, 
                   {"LOWER": {"IN": ["ca", "cancer"]}}
                ])
]

In [41]:
target_matcher = nlp.get_pipe("medspacy_target_matcher")

In [42]:
target_matcher.add(target_rules)

We then load context with the default rules and process our documents. To infer at a document level, we'll check if any entity in the document has a label of **"BREAST_CANCER"** and `is_family` is **True** but `is_negated` is **False**.

In [43]:
def get_family_history_breast_ca(doc):
    for ent in doc.ents:
        if ent.label_ != "BREAST_CANCER":
            continue
        # Check if it was family history and if it was not negated
        if ent._.is_family and not ent._.is_negated:
            return True
    return False

In [44]:
Doc.set_extension("family_history_breast_ca", getter=get_family_history_breast_ca, force=True)

In [45]:
docs = list(nlp.pipe(texts))

In [46]:
for doc in docs:
    print(doc)
    print(doc._.family_history_breast_ca)
    print()

She has a family history of breast cancer.
True

The pt's mother passed away of breast cancer several years ago.
True

The patient was diagnosed with breast cancer in 2012.
False

No family history of breast ca.
False



In [47]:
visualize_dep(docs[3])